In [1]:
%load_ext google.cloud.bigquery

In [2]:
import numpy as np 
import pandas as pd
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/shuang/Downloads/bqair-2b59936a9500.json"

# Big query helpers
from google.cloud import bigquery
from bq_helper import BigQueryHelper

# Import plotting libaries
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.io as pio
import geopandas
import shapely

# the Plotly in offline mode we can see map in the notebook.
from plotly.offline import download_plotlyjs,init_notebook_mode, plot,iplot
init_notebook_mode(connected=True)

In [3]:
import plotly
plotly.tools.set_credentials_file(username='cleopathy', api_key='Vt9pQQVXJcV8bqHarLVt')

In [4]:
# getting pm25 data:
QUERY = """
    SELECT
        state_code,
        county_code,
        date_local,
        arithmetic_mean,
        state_name,
        county_name,
        aqi
    FROM
      `bigquery-public-data.epa_historical_air_quality.pm25_frm_daily_summary`
    WHERE
      poc = 1
      AND EXTRACT(YEAR FROM date_local) in (2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017)
    
        """

In [5]:
bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")
df_pm25 = bq_assistant.query_to_pandas(QUERY) # getting data through big query to pandas
df_pm25['date_local'] = pd.to_datetime(df_pm25['date_local'])
df_pm25['year'] = df_pm25['date_local'].dt.year
df_pm25['month'] = df_pm25['date_local'].dt.month
import calendar
df_pm25['month_name'] = df_pm25['month'].apply(lambda x: calendar.month_abbr[x])
df_pm25['aqi'].fillna(df_pm25['aqi'].mean(), inplace = True)
df_pm25.head()

,state_code,county_code,date_local,arithmetic_mean,state_name,county_name,aqi,year,month,month_name
0,78,010,2004-06-02,5.4,Virgin Islands,St Croix,23.0,2004,6,Jun
1,78,010,2004-12-05,1.7,Virgin Islands,St Croix,7.0,2004,12,Dec
2,78,030,2006-10-14,7.2,Virgin Islands,St Thomas,30.0,2006,10,Oct
3,78,030,2004-06-02,7.0,Virgin Islands,St Thomas,29.0,2004,6,Jun
4,78,010,2004-01-16,2.2,Virgin Islands,St Croix,9.0,2004,1,Jan


In [8]:
df_pm25.info() 
df_pm25.drop_duplicates()
df_pm25.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1844133 entries, 0 to 44132
Data columns (total 10 columns):
state_code         object
county_code        object
date_local         datetime64[ns]
arithmetic_mean    float64
state_name         object
county_name        object
aqi                float64
year               int64
month              int64
month_name         object
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 154.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1844133 entries, 0 to 44132
Data columns (total 10 columns):
state_code         object
county_code        object
date_local         datetime64[ns]
arithmetic_mean    float64
state_name         object
county_name        object
aqi                float64
year               int64
month              int64
month_name         object
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 154.8+ MB


In [10]:
import sklearn 

In [14]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [12]:
#now we would like to predict AQI of pm25 using sklearn,but now we have to use pandas get dummies and one-hot encoder.

In [23]:
df_pm25['state_name'].value_counts()

California              201494
Pennsylvania            100662
Hawaii                   90342
Indiana                  75614
Florida                  74652
Ohio                     71064
North Carolina           59707
Georgia                  55551
Iowa                     54863
Michigan                 51259
Utah                     44904
New York                 44032
Colorado                 42329
Virginia                 41904
New Jersey               41823
Alabama                  40729
Wyoming                  38880
South Carolina           38788
Illinois                 38690
Tennessee                37970
Louisiana                34117
Texas                    33579
Kentucky                 31761
Massachusetts            28623
Wisconsin                28514
Arkansas                 27097
New Mexico               26487
Nevada                   25776
Connecticut              25011
West Virginia            24834
Maryland                 24457
Missouri                 23677
Oregon  

In [24]:
le = preprocessing.LabelEncoder()

In [25]:
le.fit(df_pm25['state_name'])

LabelEncoder()

In [17]:
list(le.classes_)

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Canada',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District Of Columbia',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Puerto Rico',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virgin Islands',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

In [26]:
state = le.transform(df_pm25['state_name'])

In [27]:
state

array([48, 48, 48, ..., 40, 40, 40])

In [28]:
sub = df_pm25[['state_code','county_code','aqi','year','month']]
sub.head()

,state_code,county_code,aqi,year,month
0,78,010,23.0,2004,6
1,78,010,7.0,2004,12
2,78,030,30.0,2006,10
3,78,030,29.0,2004,6
4,78,010,9.0,2004,1


In [ ]:
#now we will encode the year as year is non-numerical data. That way it is easier for us to predict aqi with sklearn machine learning.

In [40]:
def categorical_encoder(df):
    data_original = df[:]
    data_transformed = df[:]
    for i in df.columns:
        if df[i].dtype == object:
            le = preprocessing.LabelEncoder()
            le.fit(df[i])
            data_transformed[i] = le.transform(df[i])
            #original data is le.inverse transform
            data_original[i] = le.inverse_transform(data_transformed[i])
    return data_transformed,data_original

In [43]:
categorical_encoder(sub)

(       state_code  county_code   aqi  year  month
 0              52            5  23.0  2004      6
 1              52            5   7.0  2004     12
 2              52           17  30.0  2006     10
 3              52           17  29.0  2004      6
 4              52            5   9.0  2004      1
 5              52            5  28.0  2004      3
 6              52            5  13.0  2004     12
 7              52            5  45.0  2006      9
 8              52           17  20.0  2006      1
 9              52           17  33.0  2004      6
 10             52           17  19.0  2006      1
 11             52            5  19.0  2004      6
 12             52            5  17.0  2006      9
 13             52            5  45.0  2006     12
 14             52            5  13.0  2004     10
 15             52           17  23.0  2004      7
 16             52            5  10.0  2004      4
 17             52            5  22.0  2006     12
 18             52            5

In [ ]:
#now let's train the data: we would like to predict aqi 